In [1]:
import sys
# Assuming we are in the notebook directory add this so that we can import the library
sys.path.append('..')

import time

import numpy as np
import dask
from abcpy.core import *
from abcpy.distributions import *
from abcpy.async import wait
from abcpy.examples.ma2 import MA2, autocov, distance
from distributed import Client
from dask.dot import dot_graph
from functools import partial

import matplotlib
import matplotlib.pyplot as plt

matplotlib.style.use('ggplot')
%matplotlib inline


In [2]:
# Setup disributed
client = Client()
dask.set_options(get=client.get)

In [3]:
n = 100
t1 = 0.6
t2 = 0.2

# Set up observed data y
latents = np.random.randn(n+2)
y = MA2(n, 1, t1, t2, latents=latents)

In [4]:
# Set up the simulator
simulator = partial(MA2, n)

# Set up autocovariance summaries
ac1 = partial(autocov, 1)
ac2 = partial(autocov, 2)

# Specify the graphical model
t1 = Prior('t1', 'uniform', 0, 1)
t2 = Prior('t2', 'uniform', 0, 1)
Y = Simulator('MA2', simulator, t1, t2, observed=y)
S1 = Summary('S1', ac1, Y)
S2 = Summary('S2', ac2, Y)
d = Discrepancy('d', distance, S1, S2)

In [5]:
dists = [d.generate(1, with_values={'a': 2}) for i in range(4)]
async_dists = dists

while len(async_dists) > 0:
    r, i, async_dists = wait(async_dists, client)
    print('%f in %d' % (r, i))

print('\nResults computed at once')
client.compute(dists, sync=True)

9.122578 in 0
44.677917 in 2
11.281491 in 0
11.634260 in 0

Results computed at once


[array([[ 9.12257802]]),
 array([[ 11.28149119]]),
 array([[ 11.63426046]]),
 array([[ 44.67791678]])]